In [1]:
! pip install opensmile
! pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168842 sha256=d89a88f2a0e55570f0b3f89047bef0c178cd3f22dad0bbffac26b1dda3953f9b
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639


### Get featires values with time stamp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import opensmile
import numpy as np
import librosa

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
    #config_file_path="path_to_your_config_file.conf"
)

# Load the audio signal (replace with your own loading mechanism)
#audio_path = '/content/drive/MyDrive/FYP_dataset/new_song_feature_extraction/2.mp3'
#signal, sr= smile.process_file('/content/drive/MyDrive/FYP_dataset/CNN Features/abc.wav')
audio_path = '/content/drive/MyDrive/FYP_dataset/models/audio/dont_go_breaking_my_heart.mp3'

signal, sr = librosa.load(audio_path, sr=None)

# Segment the audio signal into 0.5-second segments
segment_duration = 2
samples_per_segment = int(segment_duration * sr)
timestamps = np.arange(0, len(signal) / sr, segment_duration)
segments = [signal[i:i + samples_per_segment] for i in range(0, len(signal), samples_per_segment)]

# Initialize a list to store features
features = []

# Extract features and timestamps for each segment
for timestamp, segment in zip(timestamps, segments):
    feature_dict = smile.process_signal(segment, sr)
    feature_dict['timestamp'] = timestamp  # Add the timestamp to the feature dictionary
    features.append(feature_dict)

# Print the extracted features and timestamps for the first segment
#print("Extracted features:", features[0])

In [29]:
import pandas as pd
import numpy as np
import glob
# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Read the feature file into a DataFrame
df = pd.read_csv(feature_files[0])

dataset_column_names = list(df.columns.drop(['Arousal_Value', 'Valence_Value', 'frameTime']))

print(dataset_column_names)
print(len(dataset_column_names))

['F0final_sma_stddev', 'F0final_sma_amean', 'voicingFinalUnclipped_sma_stddev', 'voicingFinalUnclipped_sma_amean', 'jitterLocal_sma_stddev', 'jitterLocal_sma_amean', 'jitterDDP_sma_stddev', 'jitterDDP_sma_amean', 'shimmerLocal_sma_stddev', 'shimmerLocal_sma_amean', 'logHNR_sma_stddev', 'logHNR_sma_amean', 'audspec_lengthL1norm_sma_stddev', 'audspec_lengthL1norm_sma_amean', 'audspecRasta_lengthL1norm_sma_stddev', 'audspecRasta_lengthL1norm_sma_amean', 'pcm_RMSenergy_sma_stddev', 'pcm_RMSenergy_sma_amean', 'pcm_zcr_sma_stddev', 'pcm_zcr_sma_amean', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma[0]_amean', 'audSpec_Rfilt_sma[1]_stddev', 'audSpec_Rfilt_sma[1]_amean', 'audSpec_Rfilt_sma[2]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[3]_stddev', 'audSpec_Rfilt_sma[3]_amean', 'audSpec_Rfilt_sma[4]_stddev', 'audSpec_Rfilt_sma[4]_amean', 'audSpec_Rfilt_sma[5]_stddev', 'audSpec_Rfilt_sma[5]_amean', 'audSpec_Rfilt_sma[6]_stddev', 'audSpec_Rfilt_sma[6]_amean', 'audSpec_Rfilt_sma[7

In [30]:
time_array=[]
full_song_values=[]
full_song_features=[]


for index in range(0,len(features)):

  key_list=[]
  value_list=[]

  for key, value in features[index].items():
    #print(f"Key: {key}, Value: {value[0]}")
    key_list.append(key)
    value_list.append(value[0])

  song_features_at_time=[]
  song_values_at_time=[]
  time_stamp=value_list[len(value_list)-1]
  time_array.append(time_stamp)

  for k in range(0,len(dataset_column_names)):
    for j in range(0,len(key_list)):
     if(dataset_column_names[k]==key_list[j]):
       song_features_at_time.append(key_list[j])
       song_values_at_time.append(value_list[j])
       break

  full_song_values.append( song_values_at_time)
  full_song_features.append (song_features_at_time)



In [32]:
print(time_array)
print(full_song_values)
print(full_song_features)

print(len(time_array))
print(len(full_song_values))
print(len(full_song_features))

[0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0, 36.0, 38.0, 40.0, 42.0, 44.0, 46.0, 48.0, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 62.0, 64.0, 66.0, 68.0, 70.0, 72.0, 74.0, 76.0, 78.0, 80.0, 82.0, 84.0, 86.0, 88.0, 90.0, 92.0, 94.0, 96.0, 98.0, 100.0, 102.0, 104.0, 106.0, 108.0, 110.0, 112.0, 114.0, 116.0, 118.0, 120.0, 122.0, 124.0, 126.0, 128.0, 130.0, 132.0, 134.0, 136.0, 138.0, 140.0, 142.0, 144.0, 146.0, 148.0, 150.0, 152.0, 154.0, 156.0, 158.0, 160.0, 162.0, 164.0, 166.0, 168.0, 170.0, 172.0, 174.0, 176.0, 178.0, 180.0, 182.0, 184.0, 186.0, 188.0, 190.0, 192.0, 194.0, 196.0, 198.0, 200.0, 202.0, 204.0, 206.0, 208.0, 210.0, 212.0, 214.0, 216.0, 218.0, 220.0, 222.0, 224.0, 226.0, 228.0, 230.0]
[[26.892561, 91.33681, 0.016313799, 0.7601233, 0.11117167, 0.117299505, 0.20268658, 0.1626878, 0.15554762, 0.2613995, 40.685154, -12.547605, 0.3700679, 0.6987143, 37.098923, 9.084432, 0.034547128, 0.056109596, 0.36592, 0.18455128, 15.597843, 

In [33]:
print(time_array[1])
print(full_song_values[1])
print(full_song_features[1])

print(len(full_song_values[1]))
print(len(full_song_features[1]))

2.0
[1.5514712, 87.05536, 0.021377327, 0.8039039, 0.07675002, 0.051285453, 0.041414738, 0.023003016, 0.14636484, 0.19852284, 4.399623, 9.931996, 0.20770824, 0.70974916, 0.16030584, 0.97743964, 0.01954437, 0.040608626, 0.002125177, 0.013600428, 0.013262151, 0.12196664, 0.028594092, 0.2536084, 0.07056072, 0.41017303, 0.055841707, 0.5325625, 0.06825144, 0.6397432, 0.092747554, 0.7067495, 0.10956666, 0.8076042, 0.1251929, 0.89780277, 0.20478697, 0.9947447, 0.22317007, 1.0582511, 0.1397486, 1.0440115, 0.15702543, 1.1246301, 0.32289454, 1.234999, 0.33698496, 1.2325741, 0.2526441, 1.2909242, 0.20902175, 1.3149539, 0.3032023, 1.407308, 0.34314868, 1.4497952, 0.397151, 1.4647921, 0.3769302, 1.4436944, 0.35131446, 1.3735127, 0.33591372, 1.2551095, 0.29907048, 1.0913677, 0.2412322, 0.9138478, 0.17689095, 0.7435245, 0.14793374, 0.60517997, 0.40142724, 0.37836602, 0.010737995, 0.012642492, 27.038527, 187.25961, 31.41704, 243.83012, 39.106583, 318.75, 67.75093, 455.3686, 0.25756305, 0.34819162, 24.3

## Playing Audio

In [9]:
! pip install ipython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [34]:
from IPython.display import Audio


# Play the audio
audio = Audio(audio_path)
display(audio)


Output hidden; open in https://colab.research.google.com to view.

### Predict Emotion

In [35]:
import time
from tensorflow import keras


# Load the saved Arousal and Valence models
arousal_model = keras.models.load_model('/content/drive/MyDrive/FYP_dataset/models/arousal_model.h5')  # Replace with the actual path
valence_model = keras.models.load_model('/content/drive/MyDrive/FYP_dataset/models/valence_model.h5')  # Replace with the actual path


# Categorize emotional states based on Arousal and Valence values
def categorize_emotion(arousal_value,valence_value):
    if (arousal_value > 0) and (valence_value>0):
      return "Happy"

    elif (arousal_value > 0) and (valence_value <0):
      return "Angry"

    elif (arousal_value <0) and (valence_value <0):
      return "Sad"

    elif  (arousal_value <0) and (valence_value >0):
      return "Calm"

    else:
        return "Neutral"  # Handle other cases as needed

# Function to predict Arousal and Valence for a given set of features
def predict_emotion(features):
    arousal_value = arousal_model.predict(np.array([features]))[0][0]
    valence_value = valence_model.predict(np.array([features]))[0][0]
    return arousal_value,valence_value




# Process audio features and predictions at 0.5-second intervals
for i in range(1,len(full_song_values)):


    time_stamp = time_array[i]
    song_values_at_time =full_song_values[i]


    arousal,valence = predict_emotion(song_values_at_time)
    emotion = categorize_emotion(arousal,valence)

    print(f"Timestamp: {time_stamp}    ",  f"Arousal: {arousal}   ", f"Valence: {valence}   ", f"Emotion: {emotion}")
   # print(f"Arousal: {arousal}")
    #print(f"Valence: {valence}")
    #print(f"Emotion: {emotion}")

    # You can perform further actions or analysis based on the predicted emotion
    # For example, trigger events, store results, etc.

    # Sleep for the desired time interval (0.5 seconds) until the next iteration
    time.sleep(2)


1/1 [==============================] - 0s 415ms/step
Timestamp: 2.0     Arousal: -0.7286671996116638    Valence: -0.162676602602005    Emotion: Sad
1/1 [==============================] - 0s 20ms/step
Timestamp: 4.0     Arousal: -0.2952839136123657    Valence: -0.07192429155111313    Emotion: Sad
1/1 [==============================] - 0s 19ms/step
Timestamp: 6.0     Arousal: -0.8054976463317871    Valence: -0.10981112718582153    Emotion: Sad
1/1 [==============================] - 0s 20ms/step
Timestamp: 8.0     Arousal: 0.081716388463974    Valence: -0.2293720841407776    Emotion: Angry
1/1 [==============================] - 0s 19ms/step
Timestamp: 10.0     Arousal: 0.2999331057071686    Valence: -0.0822540745139122    Emotion: Angry
1/1 [==============================] - 0s 42ms/step
Timestamp: 12.0     Arousal: -0.6788161396980286    Valence: -0.1659250259399414    Emotion: Sad
1/1 [==============================] - 0s 19ms/step
Timestamp: 14.0     Arousal: -0.7042141556739807    Val